# Algorithmic Motion Planning (236610)
John Noonan and Eli Shafer<br>
Homework 1<br>
November 2019

*QUESTION 1*: *Given sets A, B, and C formally* **Prove** *that $A \oplus (B \cup C) = (A \oplus B) \cup (A \oplus C)$*

**Proof**:

($\Longrightarrow$)

Let $p \in A \oplus (B \cup C)$.  

Then $\exists q \in A$ and $\exists r \in (B \cup C)$ s.t. $p = q + r$ (according to the definition of the Minkowski sum). 

If $r \in B$, then $A \oplus (B \cup C) = A \oplus B$.  

Or if $r \in C$, then $A \oplus (B \cup C) = A \oplus C$.  

Thus, it follows that $A \oplus (B \cup C) \subseteq (A \oplus B) \cup (A \oplus C)$.  

($\Longleftarrow$)

Suppose, now, that $s \in (A \oplus B) \cup (A \oplus C)$.

If $s \in A \oplus B$, then $\exists a \in A$ and $\exists b \in B$ s.t. $s = a + b$ (by definition of the Minkowski sum).  If $b \in B$, then $b \in B \cup C$ so that $A \oplus B \subseteq A \oplus (B \cup C)$.  

If $s \in A \oplus C$, then $\exists a \in A$ and $\exists c \in C$ s.t. $s = a + c$ (by definition of the Minkowski sum).  If $c \in C$, then $c \in C \cup B = B \cup C$ so that $A \oplus C \subseteq A \oplus (B \cup C)$.  

Either way, in light of the above, since $s \in (A \oplus B) \cup (A \oplus C)$ then $(A \oplus B) \cup (A \oplus C) \subseteq A \oplus (B \cup C)$.  

Thus, because $A \oplus (B \cup C) \subseteq (A \oplus B) \cup (A \oplus C)$ and $A \oplus (B \cup C) \supseteq (A \oplus B) \cup (A \oplus C)$, it directly follows that $A \oplus (B \cup C) = (A \oplus B) \cup (A \oplus C)$

<p style='text-align: right;'> $\blacksquare$ </p>

*QUESTION 2*: **What** is the Minkowski sum (what geometry object and what can you say about it) of (i) Two points? (ii) A point and a line? (iii) Two lines (think of all possible cases)? (iv) Two Disks?

**Answer 2**:

i) The Minkowski sum of two points is another point. If $P = \{p_1\}$ and $Q = \{q_1\}$ then $P \oplus Q$ = $\{p_1+q_1\}$.

ii) The Minkowski sum of a point and a line is the line translated by the point.  

iii) For this question and the previous one, we assume the meaning of lines is line segments.  If the lines are parallel, then the Minkowski sum is a line with the same direction. If the lines are perpendicular, then the Minkowski sum is a square.  And if the lines are non-parallel, then the Minkowski sum is a convex polygon.

iv) The Minkowski sum of two discs is a disc.  If each disc is defined by a center point and a radius, then the Minkowski sum is a disc whose center point is the sum of the two discs' center points and whose radius is the sum of the two discs' radii.

*QUESTION 3*: Recall that for the proof of Lemma. 6.2 (complexity of a trapezoidal map) we used the property that in a planar graph we have that $$E \leq 3V - 6$$
Here E and V are the number of edges and vertices in a planar graph, respectively. **Prove** Eq. 1.


**Answer 3**:

Let $F$ denote the number of faces in the planar graph.  Each edge in the planar graph is adjacent to two faces, and each face has at least 3 edges adjacent to it.  Thus, $$2E \geq 3F$$  Using Euler's formula which states that $V - E + F = 2$, we conclude the following:

$$2E \geq 3F$$
$$2E \geq 3(2 - V + E)$$
$$2E \geq 6 - 3V + 3E$$
$$-E \geq 6 - 3V$$
$$ E \leq 3V - 6$$ 

<p style='text-align: right;'> $\blacksquare$ </p>

# 3 Exact Motion Planning for a Diamond-Shaped Robot

## 3.2 Preprocessing Step (1) -- Constructing the C-space

### Computational Complexity
The input to our algorithm is: A convex polygon representing the robot with $n$ vertices, $v_1, v_2, ..., v_n$ (in our case with a diamond-shaped robot, $n = 4$) and convex polygonal obstacles with $m$ vertices, $w_1, w_2, ..., w_m$. 


#### Counter-clockwise and minimum y-coordinate:

We include a pre-processing step which ensures that the vertices are listed in counter-clockwise order and that the first of the vertices is that with the smallest y-coordinate.  We manually create the vertices for the robot using the rhombus radius and simultaneously ensure a counter-clockwise order and a first-vertex minimum y-coordinate. Thus, the pre-processing for the robot occurs in $\mathcal{O}(1)$ time.  

For the obstacle, we enforce counter-clockwise order via the `orient` method. The `orient` method has a sub-procedure which calls the `signed_area` function, which finds the signed area in linear time.  Furthermore, the `orient` function itself also performs in linear time.  Thus, the overall computational complexity of the `orient` function is $\mathcal{O}(m)$ where $m$ is the number of vertices for the obstacle.  (Please refer to the source code of the *Shapely* library for implementation details of these algorithms.)

Finding the index of the minimum y-value and making sure that the corresponding vertex is the first in the list for the obstacle is performed in linear time w.r.t. the vertices of the obstacle $\rightarrow \mathcal{O}(m)$. 

#### Minkowski Sum Computation

For reference, below is a summarized, pseudo-code version of the main part of the algorithm:

```python
i,j = 0,0
minkowski_sum = []
while i<n or j<m:
    minkowski_sum.append(v[i] + w[j])
    v_diff = v[i+1] - v[i]
    w_diff = w[j+1] - w[j]
    if (angle(v_diff) < angle(w_diff)): i += 1
    elif (angle(v_diff) > angle(w_diff)): j += 1
    else: i,j = i+1,j+1
return Polygon(minkowski_sum)
```

In our implementation, there are also checks to keep the angle in the [0, 2$\pi$] range.  As a result of the potentially alternating nature of the index increments, this implementation of the Minkowski sum operates in $\mathcal{O}(n + m)$ time.  

### Visualization of the C-space: *Provided Instance* 
![Visualization_C_space_theirs](3.2_theirs.png)

### Visualization of the C-space: *Our Instance*

In generating our own instance, we created new obstacles (and more of them), a new robot position and rhombus radius, and a new query location.  The visualization of the C-space is shown below:

![visualization_c_space_ours](3.2_ours.png)

## 3.3 Preprocessing phase (2) -- building the visibility diagram

### Computational Complexity

The input to the `get_visibility_graph` function is the list of obstacles and, optionally, the source and destination locations.  The first step of our implementation is to obtain the vertices of all of the obstacles.   
```python
v_list = [vertex for obstacle in obstacles for vertex in obstacle.exterior.coords[:-1]]
```
Let $k$ represent the total number of vertices in the graph.  If there are $l$ obstacles, each with $m$ vertices, then this computation performs in $\mathcal{O}(k) = \mathcal{O}(l\cdot m)$

We then connect each vertex to all other vertices and perform collision detection:

```python
    for i,v in enumerate(v_list):
        for j,w in enumerate(v_list[i+1:]):
            crosses = False
            line = LineString([v, w])
            for obstacle in obstacles:
                if line.within(obstacle) or line.crosses(obstacle):
                    crosses = True
                    break
            if not crosses:
                vis_graph.append(line)
```
Performing two `for` loops over the list of vertices incurs quadratic complexity in the number of vertices.  Subsequently going through all of the obstacles and performing collision detection for each obstacle incurs a cost of $\mathcal{O}(l \cdot m) = \mathcal{O}(k)$ as it is assumed that the `line.within(obstacle)` and `line.crosses(obstacle)` perform in linear time with respect to the number of vertices of the obstacle.  Thus, the overall complexity of the `get_visibility_graph` function is $\mathcal{O}(k^3)$ where $k$ is the number of vertices in the graph.

### Visualizing the Visibility Graph: *Provided Instance*
![visibility_graph_theirs](3.3_theirs.png)

### Visualizing the Visibility Graph: *Our Instance*
![visibility_graph_ours](3.3_ours.png)

## 3.4 Query phase -- computing shortest paths

### Computational Complexity
First, a side note: our implementation utilizes hash maps - Python dictionaries - for various tasks.  In light of this, when describing the computational complexity in the analysis below, we will use average case times for hash map operations.  For completeness sake, we will note here that any get/set item with the hash map incurs a cost of $\mathcal{O}(1)$ average case and $\mathcal{O}(n)$ amortized worst case. Thus, all implementation discussion below which uses the hash maps can be scaled accordingly if such consideration is desired.

The first step of the implementation is to construct an adjacency list.  More specifically, we use a hash map for the adjacency list.  Iterating through all of the edges and adding the vertices incurs a cost of $\mathcal{O}(E)$.  

Next, we construct two additional hash maps, `cost_map` and `parent_map` which map nodes to current source-to-node costs and to parent nodes.  Initializing these performs in $\mathcal{O}(V)$.  Initially all of the nodes' costs are infinity except the start node which has cost 0. 

We then construct an `unvisited` set object which contains all of the nodes.  As long as the `unvisited` object is not empty, we get the current node by finding the one with the minimum cost using the `cost_map`.  Finding the minimum cost node takes $\mathcal{O}(V)$ time but performing this inside the while loop when the `unvisited` set is not empty takes $\mathcal{O}(V^2)$ time.  The current node is removed from the `unvisited` set and then all of its adjacent nodes are explored.  For each edge, we add the current node's cost to the edge weight and check if this is less than that stored in `cost_map`. If that is the case, then we update the cost in `cost_map` and designate the current node as the *parent* to the adjacent node.  Finally, at the end, we compute the path by starting at the goal node and tracing back through the `parent_map` hash map to the start node.  This takes $\mathcal{O}(V)$ time.  

Therefore, the summarized computational complexity of our implementation is $\mathcal{O}(E + V + V^2 + E + V) = \mathcal{O}(V^2)$.


### Visualizing the Shortest Path - *Provided Instance*
![visualizing_shortest_path_theirs](3.4_theirs.png)

### Visualizing the Shortest Path - *Our Instance*
![visualizing_shortest_path_ours](3.4_ours.png)